# Applying Contextual Bandits for Recommendation systems using Tensorflow and Cloud Storage


## Learning objectives

1. Install and import required libraries.
2. Initialize and configure the MovieLens Environment.
3. Initialize the Agent.
4. Define and link the evaluation metrics.
5. Initialize & configure the Replay Buffer.
6. Setup and Train the model.
7. Inference with trained model & Tensorboard Evaluation.

## Introduction


Multi-Armed Bandit (MAB) is a Machine Learning framework in which an agent has to select actions (arms) in order to maximize its cumulative reward in the long term. In each round, the agent receives some information about the current state (context), then it chooses an action based on this information and the experience gathered in previous rounds. At the end of each round, the agent receives the reward assiociated with the chosen action.


https://www.tensorflow.org/agents/tutorials/intro_bandit#multi-armed_bandits_and_reinforcement_learning

Each learning objective will correspond to a _#TODO_ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/exercise_movielens_notebook.ipynb)

## 1. Initial Setup: installing and importing required Libraries

In [1]:
!rm -rf /opt/conda/lib/python3.7/site-packages/typing_extensions-4.4.0.dist-info
!rm -rf /opt/conda/lib/python3.7/site-packages/six-1.16.0.dist-info

In [2]:
!pip install --quiet --upgrade --force-reinstall tensorflow==2.5.0 tensorflow_probability==0.12.1 tensorflow-io==0.18.0
!pip install tf_agents==0.7.1 --quiet gast==0.3.3 --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
async-lru 2.0.4 requires typing-extensions>=4.0.0; python_version < "3.11", but you have typing-extensions 3.7.4.3 which is incompatible.
google-cloud-bigquery 3.11.4 requires grpcio<2.0dev,>=1.47.0, but you have grpcio 1.34.1 which is incompatible.
google-cloud-pubsub 2.18.4 requires grpcio<2.0dev,>=1.51.3, but you have grpcio 1.34.1 which is incompatible.
google-cloud-pubsublite 1.8.3 requires grpcio<2.0.0dev,>=1.38.1, but you have grpcio 1.34.1 which is incompatible.
grpc-google-iam-v1 0.12.6 requires grpcio<2.0.0dev,>=1.44.0, but you have grpcio 1.34.1 which is incompatible.
grpcio-status 1.48.0 requires grpcio>=1.48.0, but you have grpcio 1.34.1 which is incompatible.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.0.6 which is incompatible.
starlette 0.27.0 requires typing-extensions>

In [4]:
import functools
import os
from absl import app
from absl import flags

import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import
from tf_agents.bandits.agents import dropout_thompson_sampling_agent as dropout_ts_agent
from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents import linear_thompson_sampling_agent as lin_ts_agent
from tf_agents.bandits.agents import neural_epsilon_greedy_agent as eps_greedy_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import environment_utilities
#from tf_agents.bandits.environments import movielens_per_arm_py_environment
from tf_agents.bandits.environments import movielens_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.bandits.networks import global_and_arm_feature_network
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import time_step as ts

# If there are version / incompatibility errors, make sure you restarted the kernel and use !pip freeze in a new cell to check whether the correct TF and tf_agents version had been installed.

2023-10-09 13:53:51.274353: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [1]:
!pip freeze

absl-py==0.15.0
aiohttp @ file:///home/conda/feedstock_root/build_artifacts/aiohttp_1689804989543/work
aiohttp-cors==0.7.0
aioredis==1.3.1
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
ansiwrap==0.8.4
anyio==3.7.1
apache-beam==2.46.0
argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1692818318753/work
argon2-cffi-bindings @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi-bindings_1695386548994/work
array-record==0.4.1
arrow @ file:///home/conda/feedstock_root/build_artifacts/arrow_1662382474514/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1694046349000/work
astunparse==1.6.3
async-lru @ file:///home/conda/feedstock_root/build_artifacts/async-lru_1690563019058/work
async-timeout @ file:///home/conda/feedstock_root/build_artifacts/async-timeout_1691763562544/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1683424013410/work
Babel @ file:///home/conda/f

In [5]:
# Create target Directory if don't exist
from datetime import date
today = date.today()
fdate = date.today().strftime('%d_%m_%Y')

root_path = os.getcwd()
log_path = "{}/{}".format(root_path, fdate)
if not os.path.exists(log_path):
    os.mkdir(log_path)
    print("Directory {} Created".format(fdate))
else:    
    print("Directory {} already exists".format(fdate))

print("Full path is {}".format(log_path))

Directory 09_10_2023 Created
Full path is /home/jupyter/training-data-analyst/courses/machine_learning/deepdive2/recommendation_systems/labs/09_10_2023


## 2. Initializing and configuring the MovieLens Environment

Firstly we need to load the movielens.data csv file stored in cloud storage, load it locally and initilialze the MovielensPyenvironment with it. Refer [here](https://www.tensorflow.org/agents/api_docs/python/tf_agents/bandits/environments/movielens_py_environment/MovieLensPyEnvironment) for guidance on it.

An environment in the TF-Agents Bandits library is a class that provides observations and reports rewards based on observations and actions.

We will be using the MovieLens environment. This environment implements the MovieLens 100K dataset, available at:
  https://www.kaggle.com/prajitdatta/movielens-100k-dataset

This dataset contains 100K ratings from `m=943` users on `n=1682` items. The ratings can be organized as a matrix `A` of size `m`-by-`n`.

Note that the ratings matrix is a <b>sparse matrix</b> i.e., only a subset of certain (user, movie) pairs is provided, since not all users have seen all movies.
In order for the environment to be able to compute a reasonable estimate of the reward, which represents how much a user `i` would enjoy a movie `j`,
the environment computes a dense approximation to this sparse matrix `A`.
In collaborative filtering, it is common practice to obtain this dense approximation by means of a low-rank matrix factorization of the matrix A.

The MovieLens environment uses truncated Singular Value Decomposition (SVD) (but other matrix factorization techniques could be potentially also used).
With truncated SVD of rank `k`, the matrix `A` is factorized as follows:
$A_k = U_k \Sigma_k V_k^T$,
where:
<li>$U_k$ is a matrix of orthogonal columns of size $m$-by-$k$,<\li>
<li>$V_k$ is a matrix of orthogonal columns of size $n$-by-$k$</li>
<li> $\Sigma_k$ is a diagonal matrix of size $k$-by-$k$ that holds the $k$ largest singular values of A.</li>


By splitting $\Sigma$ into $\sqrt{\Sigma_k} \sqrt{\Sigma_k}$, we can finally approximate the matrix A as a 
product of two factors $\tilde{U}$ and $\tilde{V}$ i.e.,

$A ~= \tilde{U} \tilde{V}^T$,
where $\tilde{U} = U_k \sqrt{\Sigma_k}$ and $\tilde{V} = V_k \sqrt{\Sigma_k}$

Once the matrix factorization has been computed, the environment caches it and uses it to compute the reward for recommending an movie `j` to a user `i` 
by retrieving the (`i`, `j`)-entry of matrix $A$.
    

Apart from computing the reward when the agent recommends a certain movie to a user, the environment is also responsible for generating observations that are given as input to the agent in order to make an informed decision. In order to generate a random observation, the environment samples a random row `i` from the matrix $\tilde{U}$. Once the agent selects movie `j` then the environment responds with the (`i`, `j`)-entry of matrix $A$.

In [10]:
# initialize the movielens pyenvironment with default parameters
NUM_ACTIONS = 20 # take this as 20
RANK_K = 20 # take rank as 20
BATCH_SIZE = 8 # take batch size as 8
data_path = "gs://ta-reinforecement-learning/dataset/movielens.data" # specify the path to the movielens.data OR get it from the GCS bucket
#TODO: replace the data path if needed
env = movielens_py_environment.MovieLensPyEnvironment(
        data_path, RANK_K, BATCH_SIZE, num_movies=NUM_ACTIONS)
environment = tf_py_environment.TFPyEnvironment(env)


## 3. Initializing the Agent
Now that we have the environment query we reach the part where we define and initialize our policy and the Agent which will be our utilize that policy to make decisions given an observation. We have several policies: as shown here:

   1. [NeuralEpsilonGreedyAgent](https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870): The neural episilon greed algorithm makes a value estimate for all the arms, and then chooses the best arm with the probaility (1-episilon) and any of the random arms with a probability of epsilon. this balances the exploration-exploitation tradeoff and epsilon is set to a small value like 10%. Example: In this example we have seven arms: one of each of the classes, and if we set episilon to say 10%, then 90% of the times the agent will choose the arm with the highest value estimate ( expplotiing the one most likely to be the predicted class) and 10% of the time it will choose a random arm from all of the 7 arms( thus exploring the other possibilities). Refer [here](https://www.tensorflow.org/agents/api_docs/python/tf_agents/bandits/agents/neural_epsilon_greedy_agent/NeuralEpsilonGreedyAgent) for more information of the tensorflow agents version of the same.

    
   Each Agent is initializied with a policy: which is essentially the function approximator ( be it linear or non linear) for estimating the Q values. Ther agen trains this policy, and the policy adds the exploration-exploitation component on top of this, and also chooses the action. In this example we will use a Deep Q Network as our value function, and we use the epsilon greedy on topof this to select the actions. In this case the action space would be 20 for 20 movies, the contextual state vector would be the dense user vector from the matrix decomposition. In applied situations, a dictionary mapping could be made from a user id to its dense representation to make it more convinient for the end user.

   - Step 1. Initialize the  Qnetwork, which takes in the state and returns the value function for each action. Define the Fully connected layer parameters to be `(50, 50, 50)` from the left to the right respectively.
   - Step 2. Creating a neuron Epsilon greedy agent  with an Adam Optimizer with **Epsilon exploration value** of `0.05`, **learning rate** = `0.005`, **Dropout rate** = `0.2`. Feel free to experiment with these later to gauge their impact on the training later
   
Click [here](https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial#agent) for reference on code example of how to create a  Q network and DQN Agents
 

In [7]:
# Replace these values by reading the above instructions carefully
EPSILON = 0.05
LAYERS = None
LR = 0.005
DROPOUT_RATE = 0.2

In [11]:
# Initialize the Qnetwork
network = q_network.QNetwork(
          input_tensor_spec=environment.time_step_spec().observation,
          action_spec=environment.action_spec(),
          fc_layer_params=LAYERS)

# Creating a neuron Epsilon greedy agent with an optimizer, 
# Epsilon exploration value, learning & dropout rate
agent = eps_greedy_agent.NeuralEpsilonGreedyAgent(
  time_step_spec=environment.time_step_spec(),# get the spec/format of the environment
  action_spec=environment.action_spec(), # get the spec/format of the environment
  reward_network=network, #q network goes here
  optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR), #start w/ adam optimizer with a learning rate of .002
  epsilon=EPSILON) # we recommend an exploration of value of 1%)

2023-10-09 14:07:07.194883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-10-09 14:07:07.194945: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-09 14:07:07.194975: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20231009-080951): /proc/driver/nvidia/version does not exist
2023-10-09 14:07:07.195335: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 4. Define and link the evaluation metrics


Just like you have metrics like accuracy/recall in supervised learning, in bandits we use the [regret](https://www.tensorflow.org/agents/tutorials/bandits_tutorial#regret_metric) metric per episode. To calculate the regret, we need to know what the highest possible expected reward is in every time step. For that, we define the `optimal_reward_fn`.

Another similar metric is the number of times a suboptimal action was chosen. That requires the definition if the `optimal_action_fn`.

In [12]:
# Making functions for computing optimal reward/action and attaching the env variable to it using partial functions, so it doesnt need to be passed with every invocation
optimal_reward_fn = functools.partial(
      environment_utilities.compute_optimal_reward_with_movielens_environment,
      environment=environment)

optimal_action_fn = functools.partial(
      environment_utilities.compute_optimal_action_with_movielens_environment,
      environment=environment)


In [13]:
# Initilializing the regret and suboptimal arms metric using the optimal reward and action functions
regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)
suboptimal_arms_metric = tf_bandit_metrics.SuboptimalArmsMetric(
      optimal_action_fn)

In [14]:
step_metric = tf_metrics.EnvironmentSteps()
metrics = [tf_metrics.NumberOfEpisodes(),  #equivalent to number of steps in bandits problem
           regret_metric,  # measures regret
           suboptimal_arms_metric,  # number of times the suboptimal arms are pulled
           tf_metrics.AverageReturnMetric(batch_size=environment.batch_size)  # the average return
           ]


## 5. Initialize & configure the Replay Buffer
Reinforcement learning algorithms use replay buffers to store trajectories of experience when executing a policy in an environment. During training, replay buffers are queried for a subset of the trajectories (either a sequential subset or a sample) to "replay" the agent's experience. Sampling from the replay buffer facilitate data re-use and breaks harmful co-relation between sequential data in RL, although in contextual bandits this isn't absolutely required but still helpful.

The replay buffer exposes several functions which allow you to manipulate the replay buffer in several ways. Read more on them [here] (https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial)

In this demo we would be using the TFUniformReplayBuffer for which we need to initialize the buffer spec with the spec of the trajectory of the agent's policy, a chosen batch size( number of trajectories to store), and the maximum length of the trajectory. ( this is the amount of sequential time steps which will be considered as one data point). so a batch of 3 with 2 time steps each would result in a tensor of shape (3,2). Since unlike regular RL problems, Contextual bandits have only one time step we can keep max_length =1, however since this tutorial is to enable you for RL problems as well, let set it to 2. Do not worry, any contextual bandit agent will internally
split the time steps inside each data point such that the effective batch size ends up being (6,1). 

Create a Tensorflow based UniformReplayBuffer And initialize it with an appropriate values.
Recommended:
    **Batch size** = `8`
    **Max length** = `2` ( 2 time steps per item)

In [15]:
#TODO

STEPS_PER_LOOP = 2

# TFUniformReplayBuffer is the most commonly used replay buffer in TF-Agents. Use 'tf_uniform_replay_buffer.TFUniformReplayBuffer' to create one.
buf = tf_uniform_replay_buffer.TFUniformReplayBuffer(
      data_spec=agent.policy.trajectory_spec,
      batch_size=BATCH_SIZE,
      max_length=STEPS_PER_LOOP)

Now we have a Replay buffer but we also need something to fill it with. Often a common practice is to have 
the agent Interact with and collect experience with the environment, without actually learning from it ( i.e. only forward pass). This loop can  be either by you manually as shown [here](https://www.tensorflow.org/agents/tutorials/6_reinforce_tutorial#training_the_agent) or you can do it using the DynamicStepDriver.
The data encountered by the driver at each step is saved in a named tuple called Trajectory and broadcast to a set of observers such as replay buffers and metrics. 
This Trajectory includes the observation from the environment, the action recommended by the policy, the reward obtained, the type of the current and the next step, etc. 

In order for the driver to fill the replay buffer with data, as well as to compute ongoing metrics, it needs acess to the add_batch, functionality of the buffer, and the metrics ( both step and regular). Refer [here](https://www.tensorflow.org/agents/tutorials/5_replay_buffers_tutorial#data_collection) for more information aand example code on how initialize a step driver with observers. 


In [16]:
#TODO: setup the replay observer as a list to capture both metrics, step metrics and provide access to the function to load data from the driver into the buffer
replay_observer = [buf.add_batch, step_metric] + metrics

driver = dynamic_step_driver.DynamicStepDriver(
      env=environment,
      policy=agent.collect_policy,
      num_steps=STEPS_PER_LOOP * environment.batch_size,
      observers=replay_observer)


## 6. Setup and Train the Model

 Here we provide you a helper function in order to save your agent, the metrics and its lighter policy seperately, while training the model. We make all the aspects into trackable objects and then use checkpoint to save as well warm restart a previous training. For more information on checkpoints and policy savers ( which will be used in the training loop below) refer [here](https://www.tensorflow.org/agents/tutorials/10_checkpointer_policysaver_tutorial)

In [17]:
AGENT_CHECKPOINT_NAME = 'agent'
STEP_CHECKPOINT_NAME = 'step'
CHECKPOINT_FILE_PREFIX = 'ckpt'

In [18]:
def restore_and_get_checkpoint_manager(root_dir, agent, metrics, step_metric):
    """Restores from `root_dir` and returns a function that writes checkpoints."""
    trackable_objects = {metric.name: metric for metric in metrics}
    trackable_objects[AGENT_CHECKPOINT_NAME] = agent
    trackable_objects[STEP_CHECKPOINT_NAME] = step_metric
    checkpoint = tf.train.Checkpoint(**trackable_objects)
    checkpoint_manager = tf.train.CheckpointManager(checkpoint=checkpoint,
                                                  directory=root_dir,
                                                  max_to_keep=5)
    latest = checkpoint_manager.latest_checkpoint

    if latest is not None:
        print('Restoring checkpoint from %s.', latest)
        checkpoint.restore(latest)
        print('Successfully restored to step %s.', step_metric.result())
    else:
        print('Did not find a pre-existing checkpoint. '
                 'Starting from scratch.')
    return checkpoint_manager


In [19]:
checkpoint_manager = restore_and_get_checkpoint_manager(
  log_path, agent, metrics, step_metric)
saver = policy_saver.PolicySaver(agent.policy)
summary_writer = tf.summary.create_file_writer(log_path)
summary_writer.set_as_default()

Did not find a pre-existing checkpoint. Starting from scratch.


2023-10-09 14:18:32.275303: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Now we have all the components ready to start training the model. Here is the process for Training the model
1. We first use the DynamicStepdriver instance to collect experience( trajectories) from the environment and fill up the replay buffer.
2. We then extract all the stored experience from the replay buffer by specfiying the batch size and num_steps the same as we initialized the driver with. We extract it as tf.dataset instance.
3. We then iterate on the tf.dataset and the first sample we draw actually has all the data batch_size*num_time_steps
4. the agent then trains on the acquired experience
5. the replay buffer is cleared to make space for new data
6. Log the metrics and store them on disk
7. Save the Agent ( via checkpoints) as well as the policy

We recommend doing the training for `15,000 loops` with 2 steps per loop, and an **agent alpha** of `10.0`

In [21]:
#TODO 
# Replace `None` with the above given values
AGENT_ALPHA = 10.0
TRAINING_LOOPS = 15000

**Note:** The training will take around 50 minutes to complete and all the data are stored in the `log_path` directory. If it takes more time then click **Kernel** > **Interrupt Kernel** and proceed further.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for _ in range(TRAINING_LOOPS):
    driver.run()
    batch_size = driver.env.batch_size
    
    dataset = buf.as_dataset(
      
        sample_batch_size = BATCH_SIZE,
        num_steps=STEPS_PER_LOOP,
        single_deterministic_pass=True)

    experience, unused_info = next(iter(dataset))
    train_loss = agent.train(experience).loss
    buf.clear()
    metric_utils.log_metrics(metrics)
        # for m in metrics:
        # print(m.name, ": ", m.result())
    for metric in metrics:
        metric.tf_summaries(train_step=step_metric.result())
    checkpoint_manager.save()
    
    saver.save(os.path.join(log_path, "./", 'policy_%d' % step_metric.result()))

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
2023-10-09 14:24:16.527251: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-10-09 14:24:16.528414: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2199995000 Hz



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /home/jupyter/training-data-analyst/courses/machine_learning/deepdive2/recommendation_systems/labs/09_10_2023/./policy_16/assets


...

INFO:tensorflow:Assets written to: /home/jupyter/training-data-analyst/courses/machine_learning/deepdive2/recommendation_systems/labs/09_10_2023/./policy_20768/assets



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /home/jupyter/training-data-analyst/courses/machine_learning/deepdive2/recommendation_systems/labs/09_10_2023/./policy_20784/assets


INFO:tensorflow:Assets written to: /home/jupyter/training-data-analyst/courses/machine_learning/deepdive2/recommendation_systems/labs/09_10_2023/./policy_20784/assets


One last task before starting the training: let's upload the tensoboard logs, to get an overview of the performance of our model. We will upload our logs to `tensorboard.dev` and for that you need to 
**run the print statement below and copy the output of the cell (which is a command) into a terminal, then execute the command from there. It will give you a link from which you need to copy/paste the authentication code, and once that is done, you will receive the 
url of your model evaluation, hosted on a public [tensorboard.dev](https://tensorboard.dev/) instance**. As soon as you kicked off the training in the subsequent cell, you should see some graphs as in the picture below.

In [ ]:
print("tensorboard dev upload --logdir {} --name \"(optional) My latest experiment\" --description \"(optional) Agent trained\"".format(log_path))

<img src='./assets/example_tensorboard.png'>

## 7. Inferencing with trained model & Tensorboard Evaluation

Now that our model is trained, what if we want to determine which action to take given a new "context": for that we will iterate on our dataset to get the next item,
    make a timestep out of it by wrapping the results using ts.Timestep. It expects step_type, reward, discount, and observation as input: since we are performing prediction you can fill 
        in dummy values for the first 3: only the observation/context is relevant. Read about how it works [here](https://www.tensorflow.org/agents/api_docs/python/tf_agents/trajectories/time_step/TimeStep), and perform the task below
        
the movielens environment provides us a private observe_method which randomly samples upto 8 user context observations, and we select one of them, and reshape it to (1,20): the shape required for the model to consume.
       

In [ ]:
import numpy as np
feature = np.reshape(environment._observe()[0], (1,20))
feature.shape

In [ ]:
## Inference
step = ts.TimeStep(
        tf.constant(
            ts.StepType.FIRST, dtype=tf.int32, shape=[1],
            name='step_type'),
        tf.constant(0.0, dtype=tf.float32, shape=[1], name='reward'),
        tf.constant(1.0, dtype=tf.float32, shape=[1], name='discount'),
        tf.constant(feature,
                    dtype=tf.float64, shape=[1, 20],
                    name='observation'))

agent.policy.action(step).action.numpy()


The output of the function above recommends ( 0 indexed) movie number to recommend to the user ( represented by the user context vector). 
Read section 1 and the documentation for more clarification around this. 